# A guide for building a simple classifier based on the neural network to achieve binary classification

import the necessary python packages used in this example

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as utils_data

## 1. Generate our data

Here, we randomly generate two sets of data (x1, x2), which are points in two-dimentional space. And then, labels are given to the two sets of data. For dataset x1, the label is 0. the label of dataset x2 is 1.

In [5]:
x1 = np.random.randn(1000, 2)+5
x2 = np.random.randn(1000, 2)+7
y1 = np.zeros(x1.shape[0], dtype=int)
y2 = np.ones(x2.shape[0], dtype=int)

Here, both datasets x1 and x2 both contain 1000 data. 

Notice: The size of dataset is very important for the performance of the neural network-based model. You should be careful about it when doing your own work.

To build a complete dataset for a classifier model, we need put the two datasets together.

In [6]:
X = np.concatenate([x1, x2])  # input
Y = np.concatenate([y1, y2])  # output (target/label)

Now, we can look at the basic properties of the dataset, like the size of the dataset, the dimention of each data and the number of classes.

In [7]:
data_size, data_dim = X.shape
num_classes = int(np.max(Y)) + 1

## 2. Build our neural network-based model

First, we need to design the neural network architecture and define it in a class.

In [8]:
class MyModel(nn.Module):
    def __init__(self, input_layer_size, hidden_layer_size, num_classes):
        super(MyModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_layer_size, hidden_layer_size),
            nn.BatchNorm1d(hidden_layer_size),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(hidden_layer_size, num_classes),
        )

    def forward(self, x):
        x = self.fc1(x)
        out = self.fc2(x)
        return out

The name of the class is 'MyModel'. You can name it whatever you want.
The parameters of this class should be given in def __init__(self, *args). Here, the paramters are input_layer_size, hidden_layer_size, num_classes.
The neural network built here is a fully connected neural network with two layers. They are defined in the part of def __init__(self, *args).
The forward propagation process is achieved in the function def forward(self, x).

Notice: The architecture of the neural network is another vital aspect for the performance of the neural network-based model. You are encouraged to design and build your own network.

Now, we need to instantiate our model and set a optimizer for parameter update involved in this model. A loss function is also defined here.

In [ ]:
model = MyModel(input_layer_size=data_dim, hidden_layer_size=4, num_classes=num_classes)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_func = torch.nn.CrossEntropyLoss()

## 3. train and test

Before inputing the dataset to our model, we need to convert the type of our data from array to tensor and encapsulate it using the 'TensorDataset'. Then divide the dataset into training set and test set at the ratio of 8:2.

In [ ]:
dataset = utils_data.TensorDataset(torch.Tensor(X), torch.LongTensor(Y))
train_size = int(data_size * 0.8)
test_size = data_size - train_size
train_set, test_set = utils_data.random_split(dataset, [train_size, test_size])
train_loader = utils_data.DataLoader(dataset=train_set, batch_size=8, shuffle=True)
test_loader = utils_data.DataLoader(dataset=test_set, batch_size=8, shuffle=True)

The following is the training and testing process in detail.

In [ ]:
best_accuracy = 0.0
max_epoch = 100  # the maximum epoch for training
for epoch in range(max_epoch):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        batch_output = model(batch_x)
        batch_loss = loss_func(batch_output, batch_y)

        optimizer.zero_grad()  # gradient is set to 0
        batch_loss.backward()  # loss is back propagated
        optimizer.step()       # update the model's parameters

        # calculate loss
        train_loss += batch_loss.item()
        # calculate accuracy
        _, batch_predicted = torch.max(batch_output, dim=1)
        train_acc += (batch_predicted == batch_y).sum().item()

    train_loss /= (step+1)
    train_acc /= train_size

    # test accuracy
    model.eval()
    test_acc = 0.0
    for test_x, test_y in test_loader:
        test_output = model(test_x)
        _, test_predicted = torch.max(test_output, dim=1)
        test_acc += (test_predicted == test_y).sum().item()
    test_acc /= test_size

    print('epoch={:d}\ttrain loss={:.6f}\ttrain accuracy={:.3f}\ttest accuracy={:.3f}'.format(
        epoch, train_loss, train_acc, test_acc))

    if test_acc >= best_accuracy:
        torch.save(model.state_dict(), 'classifier.pkl')  # the current parameters of the model are saved in the file 'classifier.pkl'
        best_accuracy = test_acc

Notice: The maximum epoch for training is an adjustable parameter, which also has an effect on the performance of the model.

Here, we give a visualization display for the two dataset. The red and green points respectively represent the class 0 and class 1. The training data and test data are shown as solid points and squre points. The prediction results are displied using triangle.

In [ ]:
# visualization
plt.figure(figsize=(12, 10))

for train_x, train_y in train_loader:
    train_x = np.array(train_x.data)
    train_y = np.array(train_y.data)
    index_class_0 = train_y == 0
    index_class_1 = train_y == 1

    p1, = plt.plot(train_x[index_class_0, 0], train_x[index_class_0, 1], 'ro', alpha=0.3)
    p2, = plt.plot(train_x[index_class_1, 0], train_x[index_class_1, 1], 'go', alpha=0.3)

for test_x, test_y in test_loader:
    test_output = model(test_x)

    test_x = np.array(test_x.data)
    test_y = np.array(test_y.data)
    index_class_0 = test_y == 0
    index_class_1 = test_y == 1
    p3, = plt.plot(test_x[index_class_0, 0], test_x[index_class_0, 1], 'rs')
    p4, = plt.plot(test_x[index_class_1, 0], test_x[index_class_1, 1], 'gs')

    _, test_predicted = torch.max(test_output, dim=1)
    test_predicted = np.array(test_predicted.data)
    pred_class_0 = test_predicted == 0
    pred_class_1 = test_predicted == 1
    p5, = plt.plot(test_x[pred_class_0, 0], test_x[pred_class_0, 1], 'r^')
    p6, = plt.plot(test_x[pred_class_1, 0], test_x[pred_class_1, 1], 'g^')

plt.legend([p1, p2, p3, p4, p5, p6], ['class 0: train data', 'class 1: train data',
                                      'class 0: test data', 'class 1: test data',
                                      'class 0: prediction', 'class 1: prediction'])